In [3]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key:  ········


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
# Step 1: Initialize Gemini 1.5 Pro
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

template = "tell me a joke about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)

topic = input("give a topic")
chain = prompt_template | llm | StrOutputParser()
print(chain.invoke(topic))




give a topic cats


Why was the cat sitting on the computer? 

To keep an eye on the mouse!


In [14]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


books_dir = r"C:\Users\Krish\Downloads\books"
persistent_directory = r"C:\Users\Krish\Downloads\chroma_db"
os.makedirs(persistent_directory, exist_ok=True)


print("\n--- Creating embeddings ---")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("--- Finished creating embeddings ---")


if os.path.exists(os.path.join(persistent_directory, "index")):
    print("\n--- Loading existing vector store ---")
    db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)
else:
    print("\n--- Creating new vector store ---")
    

    documents = []
    book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

    for book_file in book_files:
        file_path = os.path.join(books_dir, book_file)
        loader = TextLoader(file_path, encoding="utf-8")
        book_docs = loader.load()
        for doc in book_docs:
            doc.metadata = {"source": book_file} 
            documents.append(doc)


    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[0].page_content}\n")

    db = Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)
    print("--- Finished creating and saving vector store ---")

query = "in which army did Elias fight when he migra"
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}
)
relevant_docs = retriever.invoke(query)

print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\n\nRelevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nPlease provide an answer based only on the provided documents. If the answer is not found in the documents, respond with 'I'm not sure'."
)
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

print("Start chatting with the AI! Type 'exit' to end the conversation.")
chat_history=[]
while True:
    query = input("You: ")
    if query.lower() in {"exit", "quit"}:
        print("Ending chat.")
        break
    retrieved_docs = history_aware_retriever.invoke({"chat_history": chat_history, "input": query})
    answer = question_answer_chain.invoke({"input": query, "chat_history": chat_history, "context": retrieved_docs})
    print(f"AI: {answer}")
    print("Sources:")
    for doc in retrieved_docs:
        print(f"- {doc.metadata.get('source', 'Unknown')}")
    chat_history.append(HumanMessage(content=query))
    chat_history.append(SystemMessage(content=answer))



--- Creating embeddings ---


Created a chunk of size 1147, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1356, which is longer than the specified 1000
Created a chunk of size 1257, which is longer than the specified 1000
Created a chunk of size 1385, which is longer than the specified 1000
Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1128, which is longer than the specified 1000
Created a chunk of size 1406, which is longer than the specified 1000
Created a chunk of size 1038, which is longer than the specified 1000
Created a chunk of size 1022, which is longer than the specified 1000
Created a chunk of size 1428, which is longer than the specified 1000
Created a chunk of size 1539, which is longer than the specified 1000
Created a chunk of s

--- Finished creating embeddings ---

--- Creating new vector store ---


Created a chunk of size 2086, which is longer than the specified 1000
Created a chunk of size 1121, which is longer than the specified 1000
Created a chunk of size 1366, which is longer than the specified 1000
Created a chunk of size 1011, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 1219, which is longer than the specified 1000
Created a chunk of size 1875, which is longer than the specified 1000
Created a chunk of size 1307, which is longer than the specified 1000
Created a chunk of size 2271, which is longer than the specified 1000
Created a chunk of size 1430, which is longer than the specified 1000
Created a chunk of size 1763, which is longer than the specified 1000
Created a chunk of size 1575, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of size 1149, which is longer than the specified 1000
Created a chunk of s


--- Document Chunks Information ---
Number of document chunks: 1699
Sample chunk:
﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release date: March 1, 1999 [eBook #1661]
                Most recently updated: October 10, 2023

Language: English

Credits: an anonymous Project Gutenberg volunteer and Jose Menendez


*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***


The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Cont

You:  how did romeo die?


AI: This excerpt doesn't describe Romeo's death, but it does show he is exiled for killing Tybalt.  Juliet grieves and alternates between calling for Tybalt and Romeo.  Romeo then speaks to Friar Laurence, distraught over the impact of his actions on Juliet.
Sources:
- romeo_and_juliet.txt
- romeo_and_juliet.txt
- romeo_and_juliet.txt
- romeo_and_juliet.txt
- romeo_and_juliet.txt


NameError: name 'ans' is not defined